Day 7: Figuring out directory sizes from command line input/output (ugh)

In [3]:
input = []
with open('day7-input.txt') as f:
    for line in f.readlines():
        input.append(line.rstrip())

In [5]:
def is_command(string):
    return string[0] == "$"

def is_calling_ls(string):
    return string[2:4] == "ls"

def is_a_dir(string):
    return string[0:3] == "dir"

def is_calling_cd(string):
    return string[2:4] == "cd"

def has_size(string):
    return string[0] in [*'0123456789']

is_calling_ls('$ ls'), is_command("$   a sd as d as"), is_a_dir('dir drblq'), is_calling_cd('$ cd /'), has_size('43251 jw.jpg'), has_size('$ asdas')

(True, True, True, True, True, False)

Plan:

Track all the way down where we are.

In [20]:
ddd = {"a":"str", "b":4}
"a" not in ddd.keys()

False

In [55]:
dirs_contents = {}
current_path = []

def path_gen(path_list):
    return "-".join(path_list)

for line in input:
    lsplit = line.split()
    if is_calling_cd(line):
        in_ls = False
        where = lsplit[2]
        if where == '..': current_path.pop()
        else: current_path.append(where)
        
    if in_ls:
        tmp_path = path_gen(current_path)
        if tmp_path not in dirs_contents.keys(): 
            dirs_contents[tmp_path] = []
        if has_size(line):
            dirs_contents[tmp_path].append(int(lsplit[0]))  # if it's a file add its size to that working directory's contents
        elif is_a_dir(line):
            subdir_path = path_gen(current_path)
            tmp_subdir = lsplit[1]
            subdir_path += "-"
            subdir_path += tmp_subdir
            dirs_contents[tmp_path].append(subdir_path)  # if it's a subdir, add it's full path
        elif is_command(line):
            in_ls = False  # no longer in the ls command output
        else:
            print('panic1')
    elif is_calling_ls(line):
        in_ls = True

dirs_contents  # ok!

{'/': ['/-drblq',
  133789,
  '/-jpfrhmw',
  '/-jqfwd',
  '/-ncgffsr',
  12962,
  '/-qnbq',
  '/-rqdngnrq',
  '/-shcvnqq',
  '/-vsd',
  '/-vtzvf'],
 '/-drblq': [133843, '/-drblq-brfnfhj', 268201, 80676],
 '/-drblq-brfnfhj': [150447, '/-drblq-brfnfhj-nhvgrzs'],
 '/-drblq-brfnfhj-nhvgrzs': [282889,
  19004,
  '/-drblq-brfnfhj-nhvgrzs-vbzr',
  6338,
  '/-drblq-brfnfhj-nhvgrzs-wdcn'],
 '/-drblq-brfnfhj-nhvgrzs-vbzr': [225101, 243277],
 '/-drblq-brfnfhj-nhvgrzs-wdcn': [154089],
 '/-jpfrhmw': [87622, 26165, '/-jpfrhmw-vbzr'],
 '/-jpfrhmw-vbzr': ['/-jpfrhmw-vbzr-blhstw',
  16919,
  '/-jpfrhmw-vbzr-rgdp',
  116477,
  242592,
  '/-jpfrhmw-vbzr-vbzr',
  '/-jpfrhmw-vbzr-wmct'],
 '/-jpfrhmw-vbzr-blhstw': [98023],
 '/-jpfrhmw-vbzr-rgdp': ['/-jpfrhmw-vbzr-rgdp-gcb',
  141507,
  '/-jpfrhmw-vbzr-rgdp-ssvzm'],
 '/-jpfrhmw-vbzr-rgdp-gcb': [189016],
 '/-jpfrhmw-vbzr-rgdp-ssvzm': [82667],
 '/-jpfrhmw-vbzr-vbzr': [120202, 86205],
 '/-jpfrhmw-vbzr-wmct': ['/-jpfrhmw-vbzr-wmct-fbsfcgph',
  155709,
  13636,
 

In [50]:
dir_values = {key:0 for (key,value) in dirs_contents.items()} # can keep track of storage used in each dir in this perhaps

In [78]:
all_the_keys_list = list(dir_values.keys())  # USING SETS GOT THE WRONG ANSWER ?!?!??!?!?

while len(all_the_keys_list) > 0:
    keys_to_remove = []
    for k in all_the_keys_list:
        # LOOK IN A DIR, TRY TO ADD UP, IF WE CANT (SUBDIR'S VALUE UNKNOWN), MOVE ON
        contents = dirs_contents[k] # list of all inside current dir
        running_total = 0
        for c in contents:
            did_it = True
            if type(c) == int:
                running_total += c
            else:
                val = dir_values[c]  # found a subdir, do we know its value?
                if val == 0:
                    did_it = False
                else:
                    running_total += val
        # So if we managed to figure out the size then we add it to the values dir
        if did_it:
            dir_values[k] = running_total
            running_total = 0
            keys_to_remove.append(k)
    for remove_this_key in keys_to_remove:
        #all_the_keys.remove(remove_this_key)  <--- this was a set, but looping through the set, and checking that length overshot the answer. Wonder why :think:
        all_the_keys_list.remove(remove_this_key)
    # DID WE GET THROUGH THEM?    

In [79]:
final_totals = [val for (key,val) in dir_values.items() if val <= 100000]
sum(final_totals)

1367870

Part 2: Which dir to delete

total size =  70000000
needed size >= 30000000

In [86]:
SYS_TOTAL = 70000000
NEED = 30000000

outermost_dir_space = dir_values['/']
remaining = SYS_TOTAL - outermost_dir_space
have_to_del = NEED - remaining
have_to_del

528671

In [96]:
could_del = {key:space for (key,space) in dir_values.items() if space >= have_to_del}
least_space = min(could_del.values())
least_space, list(could_del.keys())[list(could_del.values()).index(least_space)] # if we had to know where!

(549173, '/-vsd-ncgffsr-hjgm-ljqjtdmf')